In [2]:
import uproot 
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.spatial.transform import Rotation as R
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from pylorentz import Momentum4
from pylorentz import Position4

In [3]:
df_tt = pd.read_pickle('./df_tt.pkl')
df_rho = df_tt[(df_tt['mva_dm_1']==1) & (df_tt['mva_dm_2']==1) & (df_tt["tau_decay_mode_1"] == 1) & (df_tt["tau_decay_mode_2"] == 1)]
# drop unnecessary labels 
df = df_rho.drop(["mva_dm_1","mva_dm_2","tau_decay_mode_1","tau_decay_mode_2", "wt_cp_sm", "wt_cp_ps", "wt_cp_mm", "rand"], axis=1).reset_index(drop=True)

In [4]:
df.head()

,aco_angle_1,aco_angle_5,aco_angle_6,aco_angle_7,ip_x_1,ip_y_1,ip_z_1,ip_x_2,ip_y_2,ip_z_2,...,sv_z_1,sv_x_2,sv_y_2,sv_z_2,gen_nu_p_1,gen_nu_phi_1,gen_nu_eta_1,gen_nu_p_2,gen_nu_phi_2,gen_nu_eta_2
0,2.657029,1.509938,3.327978,4.475070,-0.003927,-0.001750,0.006455,0.003345,0.000417,-0.012070,...,0.0,0.0,0.0,0.0,11.907320,-1.878383,-0.443223,12.628682,1.261835,0.048225
1,1.547246,3.710838,4.023599,1.860006,-0.001310,0.006071,0.004181,-0.007126,0.006137,-0.004593,...,0.0,0.0,0.0,0.0,14.349887,2.519850,-0.934011,44.844142,-0.313698,-1.423014
2,3.750274,3.446808,2.489589,2.793055,-0.009182,-0.004744,0.006723,-0.010568,0.009932,-0.005184,...,0.0,0.0,0.0,0.0,35.673293,0.106444,1.180579,23.342057,-2.873663,1.126849
3,1.192612,2.157516,2.071870,1.106966,0.011873,0.001333,0.005862,-0.027225,-0.009527,0.018453,...,0.0,0.0,0.0,0.0,2.964097,-0.686183,-1.008616,0.729467,2.953007,-0.809572
4,3.264792,2.106487,0.394414,1.552719,-0.003312,0.001791,0.009937,0.003434,-0.000035,0.014993,...,0.0,0.0,0.0,0.0,8.663849,-2.653995,-0.131161,27.029359,0.369547,-0.154201


In [5]:
# pi_1_boosted_rot = np.load('gen_info/pi_1_boosted_rot.npy', allow_pickle=True)
# pi_2_boosted_rot = np.load('gen_info/pi_2_boosted_rot.npy', allow_pickle=True)
# pi0_1_boosted_rot = np.load('gen_info/pi0_1_boosted_rot.npy', allow_pickle=True)
# pi0_2_boosted_rot = np.load('gen_info/pi0_2_boosted_rot.npy', allow_pickle=True)
# rho_1_boosted_rot = pi_1_boosted_rot + pi0_1_boosted_rot
# rho_2_boosted_rot = pi_2_boosted_rot + pi0_2_boosted_rot

In [6]:
pi_1 = Momentum4(df['pi_E_1'], df["pi_px_1"], df["pi_py_1"], df["pi_pz_1"])
pi_2 = Momentum4(df['pi_E_2'], df["pi_px_2"], df["pi_py_2"], df["pi_pz_2"])
pi0_1 = Momentum4(df['pi0_E_1'], df["pi0_px_1"], df["pi0_py_1"], df["pi0_pz_1"])
pi0_2 = Momentum4(df['pi0_E_2'], df["pi0_px_2"], df["pi0_py_2"], df["pi0_pz_2"])
rho_1 = pi_1 + pi0_1
rho_2 = pi_2 + pi0_2
ip_1 = np.c_[df['ip_x_1'], df['ip_y_1'], df['ip_z_1']]
ip_2 = np.c_[df['ip_x_2'], df['ip_y_2'], df['ip_z_2']]

to do:  
- [ ] change units - ip is in cm

In [17]:
l_tau = 87e-4 # in cm
# testing sandbox
for i in range(100):
    vis_dir = rho_1[:,i][1:]/np.linalg.norm(rho_1[:,i][1:])
    ip = ip_1[i]
    delta = vis_dir.dot(ip)**2 - (np.linalg.norm(ip)**2-l_tau**2)
    d_plus = -vis_dir.dot(ip) + delta
    d_minus = -vis_dir.dot(ip) - delta
    # todo: transform d to a point
    if delta > 0:
        sv_1 = d_plus*vis_dir
        sv_2 = d_minus*vis_dir
    else:
        pass

[-6.87516835e-06 -1.87633530e-05 -9.29102979e-06] [2.82025532e-06 7.69689460e-06 3.81126322e-06]
[ 5.06331838e-05 -3.31152934e-05  6.17658701e-05] [ 7.36489327e-05 -4.81681347e-05  8.98420772e-05]
[-2.51972242e-05 -3.06006234e-06  3.21009778e-05] [ 3.09885496e-05  3.76338651e-06 -3.94790607e-05]
[ 8.05809544e-06 -8.86320135e-05  1.23474989e-04] [ 1.27371088e-05 -1.40097075e-04  1.95171972e-04]
[ 5.16328890e-06 -1.65629484e-05  2.12300019e-05] [ 3.26599648e-05 -1.04767585e-04  1.34288653e-04]
[ 2.92149944e-05 -3.24669115e-05 -1.55479868e-05] [ 1.05014072e-04 -1.16703174e-04 -5.58876507e-05]
[-2.91400473e-05  1.90453551e-06 -5.94625730e-06] [ 7.69064532e-05 -5.02645275e-06  1.56933705e-05]
[ 1.02786046e-04 -1.08517745e-04  5.56497483e-06] [ 2.38334491e-05 -2.51624833e-05  1.29037501e-06]
[ 7.57476393e-05 -7.84726342e-05  1.61444603e-04] [ 2.19426819e-05 -2.27320623e-05  4.67674980e-05]
[-1.83607179e-05 -5.92820616e-06  6.78290598e-06] [-1.25634665e-04 -4.05642197e-05  4.64125708e-05]
[ 1

1.5538790369533218e-05